In [1]:
import pandas as pd
import numpy as np
from time import time
import re
import string
import socket
from glob import glob

In [2]:
country_code = "US"
print('Country:', country_code)

# Local
if 'samuel' in socket.gethostname().lower():
    path_to_data = os.path.join('../../data/classification',country_code)
# Cluster
else:
    path_to_data = os.path.join('/scratch/spf248/twitter/data/classification',country_code)
print('Path to data:',path_to_data)

Country: US
Path to data: ../../data/classification/US


# Training Set

In [110]:
# Only keep one label per worker and tweet
labels=pd.concat(
[pd.read_csv(file) for file in glob(os.path.join(path_to_data,'labeled','*','labels.csv'))]).sort_values(
by=['tweet_id','class_id','QIDWorker']).drop_duplicates(
['tweet_id','class_id','QIDWorker']).set_index(
['tweet_id','class_id','QIDWorker'])

print('# labels:', labels.shape[0])

# labels: 45825


In [112]:
def is_labeled(x):
    # If First sequence was allocated more than once
    if x[0]>1:
        # If no other sequence
        if len(x)==1:
            return True
        else:
            # If second sequence less popular
            if x[1]<x[0]:
                return True
    return False

# Counts of label sequences for each tweet
counts=labels.groupby(['tweet_id','QIDWorker'])['score'].apply(
lambda x:'_'.join(x)).groupby('tweet_id').value_counts().rename('count')

# Keep Tweets that were labeled more than once with most popular sequence of labels strictly dominating
ids_labeled=counts.groupby('tweet_id').apply(list).apply(is_labeled).where(lambda x:x==True).dropna().index
print('# labeled tweets:', len(ids_labeled))

# labeled tweets: 2296


In [113]:
# Keep most popular label sequence
labels=counts.reindex(ids_labeled,level='tweet_id').reset_index(
level='score').groupby('tweet_id')['score'].first().apply(
lambda x:pd.Series(x.split('_'),index=range(1,6)))

In [114]:
labels.to_csv(os.path.join(path_to_data,'labeled','labels.csv'))

In [115]:
class2name=dict(zip([str(x) for x in range(1,6)],[
'Does this tweet indicate that the user is currently unemployed?',
'Does this tweet indicate that the user became unemployed within the last month?',
'Does this tweet indicate that the user is currently searching for a job?',
'Does this tweet indicate that the user was hired within the last month?',
'Does this tweet contain a job offer?',
]))

class2name

{'1': 'Does this tweet indicate that the user is currently unemployed?',
 '2': 'Does this tweet indicate that the user became unemployed within the last month?',
 '3': 'Does this tweet indicate that the user is currently searching for a job?',
 '4': 'Does this tweet indicate that the user was hired within the last month?',
 '5': 'Does this tweet contain a job offer?'}

In [116]:
labels

,1,2,3,4,5
tweet_id,,,,,
278519817328279552,yes,no,yes,no,no
278988941837873152,yes,no,no,no,no
279004515271864320,no,no,no,no,no
280411569995776002,no,no,no,no,no
281540898641285120,yes,yes,no,no,no
...,...,...,...,...,...
1072914627594514434,yes,no,no,no,no
1073295386511589377,yes,no,no,no,no
1077327152817684480,no,no,no,no,yes
